In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# create some arrays in numpy
n=10
X = np.linspace(0,10,n)
Y = np.random.normal(size=(n,1))


## Running in eager mode

In [6]:

#some complex tensorflow code we want to repeat with different values
X_in = tf.expand_dims(X,-1)
Xs = tf.square(X_in)
Xs = tf.reduce_sum(Xs, axis=-1, keepdims=True)
dist = -2 * tf.matmul(X_in, X_in, transpose_b=True)
dist += Xs + tf.linalg.matrix_transpose(Xs)
K = tf.exp(-dist)
K = K + (tf.eye(tf.shape(K)[0], dtype=tf.float64) * 1e-6)
L = tf.linalg.cholesky(K)
f = tf.matmul(L,Y)
# end of the complex code

In [3]:
X

array([ 0.        ,  1.11111111,  2.22222222,  3.33333333,  4.44444444,
        5.55555556,  6.66666667,  7.77777778,  8.88888889, 10.        ])

In [4]:
Y

array([[-2.2784691 ],
       [-0.57883089],
       [-1.27917737],
       [ 0.86466702],
       [-0.94556172],
       [-0.10021122],
       [-1.75358708],
       [-1.08452046],
       [-1.60878601],
       [-0.06349697]])

In [8]:
#Run the code with a timer - EAGER MODE
import time

def myfunction(X_in,Y_in):
    # some complex tensorflow code we want to repeat with different values
    X_in = tf.expand_dims(X_in,-1)
    Xs = tf.square(X_in)
    Xs = tf.reduce_sum(Xs, axis=-1, keepdims=True)
    dist = -2 * tf.matmul(X_in, X_in, transpose_b=True)
    dist += Xs + tf.linalg.matrix_transpose(Xs)
    K = tf.exp(-dist)
    K = K + (tf.eye(tf.shape(K)[0], dtype=tf.float64) * 1e-6)
    L = tf.linalg.cholesky(K)
    f = tf.matmul(L,Y_in)
    # end of the complex code
    return f

# start the timer
start = time.time()
# run the code once
Y = np.random.normal(size=(n,1))
f_function = myfunction(X,Y)
# report the time
print('the code took ', time.time()-start, ' seconds to run')

the code took  0.0041065216064453125  seconds to run


In [10]:
# start the timer for a loop of function runs
start = time.time()
for i in range(5000):
    Y = np.random.normal(size=(n,1))
    f_function = myfunction(X,Y)
# report the time
print('the loop took ', time.time()-start, ' seconds to run')

the loop took  11.056743860244751  seconds to run


## Running in graph mode

In [11]:
# create the function using the function decorator  - GRAPH MODE
@tf.function
def myfunction(X_in,Y_in):
    # some complex tensorflow code we want to repeat with different values
    X_in = tf.expand_dims(X_in,-1)
    Xs = tf.square(X_in)
    Xs = tf.reduce_sum(Xs, axis=-1, keepdims=True)
    dist = -2 * tf.matmul(X_in, X_in, transpose_b=True)
    dist += Xs + tf.linalg.matrix_transpose(Xs)
    K = tf.exp(-dist)
    K = K + (tf.eye(tf.shape(K)[0], dtype=tf.float64) * 1e-6)
    L = tf.linalg.cholesky(K)
    f = tf.matmul(L,Y_in)
    # end of the complex code
    return f

# start the timer
start = time.time()

# run the code once
Y = np.random.normal(size=(n,1))
f_function = myfunction(X,Y)

# report the time
print('the code took ', time.time()-start, ' seconds to run')

# start the timer for a loop of function runs
start = time.time()
for i in range(5000):
    Y = np.random.normal(size=(n,1))
    f_function = myfunction(X,Y)
# report the time
print('the loop took ', time.time()-start, ' seconds to run')

the code took  0.26236581802368164  seconds to run
the loop took  4.307241678237915  seconds to run


In [ ]:
# What has happened now is that the code takes far longer to run the first time but is then more efficient
#after that